#DATA ANALYTICS

## Importing Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import gc
import time
import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm
from random import randint

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import calendar
import datetime
from matplotlib.colors import ColorConverter, ListedColormap
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad
from statsmodels.tsa.seasonal import seasonal_decompose

import scipy
from scipy.fftpack import fft
import statsmodels
from scipy import signal
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from fbprophet import Prophet
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

%matplotlib inline

##Reading Data

In [ ]:
data = pd.read_excel('Train_data.xlsx')
data.head()

,Warehouse id,Region,SKU id,2018-04-01 00:00:00,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-07-01 00:00:00,2018-08-01 00:00:00,2018-09-01 00:00:00,2018-10-01 00:00:00,...,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00
0,Wh-1,NORTH,SKU-1,27,13,30,14,22,8,8,...,12,8,8,8,8,8,14,16,8,12
1,Wh-1,NORTH,SKU-2,88,28,127,130,58,10,7,...,16,7,7,7,7,7,13,43,13,43
2,Wh-1,NORTH,SKU-3,6,6,6,6,6,6,6,...,8,6,6,6,6,6,6,8,26,6
3,Wh-1,NORTH,SKU-4,162,184,110,172,90,40,18,...,82,78,38,18,22,28,36,352,58,18
4,Wh-1,NORTH,SKU-5,171,402,627,615,300,417,141,...,468,216,75,6,30,66,141,699,183,24


##Data Preprocessing

In [ ]:
dataN = data[data['Region']== 'NORTH']
dataS = data[data['Region']== 'EAST']
dataE = data[data['Region']== 'SOUTH']
dataW = data[data['Region']== 'WEST']
# print(data.columns)
date_cols = [i for i in data.columns if type(i) == datetime.datetime]
# print(date_cols)
dataN.head()

,Warehouse id,Region,SKU id,2018-04-01 00:00:00,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-07-01 00:00:00,2018-08-01 00:00:00,2018-09-01 00:00:00,2018-10-01 00:00:00,...,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00
0,Wh-1,NORTH,SKU-1,27,13,30,14,22,8,8,...,12,8,8,8,8,8,14,16,8,12
1,Wh-1,NORTH,SKU-2,88,28,127,130,58,10,7,...,16,7,7,7,7,7,13,43,13,43
2,Wh-1,NORTH,SKU-3,6,6,6,6,6,6,6,...,8,6,6,6,6,6,6,8,26,6
3,Wh-1,NORTH,SKU-4,162,184,110,172,90,40,18,...,82,78,38,18,22,28,36,352,58,18
4,Wh-1,NORTH,SKU-5,171,402,627,615,300,417,141,...,468,216,75,6,30,66,141,699,183,24


##Data Visualisation and Exploratory Data Analysis

###Region-Wise SKU id's

In [ ]:
n, _ = dataN.shape
s, _ = dataS.shape
e, _ = dataE.shape
w, _ = dataW.shape
fig = px.pie(values=[n,s,e,w], names=["North", "South", "East", "West"], title="Region-Wise Total SKUs")
fig.show()

###Region-Wise Aggregate Sales

In [ ]:
sum1n = dataN.sum()
sum2n = pd.DataFrame(sum1n[3:])
sum3n = sum2n.sum()
north_sum = sum3n[0]
sum1s = dataS.sum()
sum2s = pd.DataFrame(sum1s[3:])
sum3s = sum2s.sum()
south_sum = sum3s[0]
sum1e = dataE.sum()
sum2e = pd.DataFrame(sum1e[3:])
sum3e = sum2e.sum()
east_sum = sum3e[0]
sum1w = dataW.sum()
sum2w = pd.DataFrame(sum1w[3:])
sum3w = sum2w.sum()
west_sum = sum3w[0]
west_sum
random_x = [north_sum,south_sum,east_sum,west_sum]
names = ['North','South','East','West']
 
fig = px.pie(values=random_x, names=names,title = "Region-wise aggregrate sales")
fig.show()

###Plot Name??

In [ ]:
dic = {'N':'SKU-9','E':'SKU-53','W':'SKU-219','S':'SKU-215'}
x_1 = dataN[dataN['SKU id'] == dic['N']][date_cols].values[0]
# print(x_1)
x_2 = dataN.loc[data['SKU id'] == dic['E']][date_cols].values[0]
x_3 = dataN.loc[data['SKU id'] == dic['W']][date_cols].values[0]
x_4 = dataN.loc[data['SKU id'] == dic['S']][date_cols].values[0]
smoothened_signal_1 = signal.savgol_filter(x_1, polyorder = 3, window_length=15)
smoothened_signal_2 = signal.savgol_filter(x_2, polyorder = 3, window_length=15)
smoothened_signal_3 = signal.savgol_filter(x_3, polyorder = 3, window_length=15)
smoothened_signal_4 = signal.savgol_filter(x_4, polyorder = 3, window_length=15)



In [ ]:
fig = make_subplots(rows=4, cols=1,subplot_titles=("N SKU-9", "E SKU-53", "W SKU-219", "S SKU-215"))


fig.add_trace(go.Scatter(x=np.arange(len(x_1)), y=x_1, showlegend=False,
                    mode='lines+markers', name="N SKU-9", marker=dict(color="limegreen")), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(len(x_1)), y=smoothened_signal_1, showlegend=False,
                    mode='lines', name="smoothened signal", marker=dict(color="darkgreen")), row=1, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_2)), y=x_2, showlegend=False,
                    mode='lines+markers', name=" E SKU 53", marker=dict(color="hotpink")), row=2, col=1)
fig.add_trace(go.Scatter(x=np.arange(len(x_2)), y=smoothened_signal_2, showlegend=False,
                    mode='lines', name="smoothened signal", marker=dict(color="darkred")), row=2, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_3)), y=x_3, showlegend=False,
                    mode='lines+markers', name="W SKU 219", marker=dict(color="lightsteelblue")), row=3, col=1)
fig.add_trace(go.Scatter(x=np.arange(len(x_3)), y=smoothened_signal_3, showlegend=False,
                    mode='lines', name="smoothened signal", marker=dict(color="darkslateblue")), row=3, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_4)), y=x_4, showlegend=False,
                    mode='lines+markers', name="S SKU 215", marker=dict(color="orange")), row=4, col=1)
fig.add_trace(go.Scatter(x=np.arange(len(x_3)), y=smoothened_signal_4, showlegend=False,
                    mode='lines', name="smoothened signal", marker=dict(color="yellow")), row=4, col=1)

fig.update_layout(height=1200, width=800, title_text="Sales Snippets from One Shop in Each Region")
fig.show()

###Graphs of Aggregate Sales - Overall, Region-wise

In [ ]:
aggr_array = []
for d in date_cols:
    aggr_array.append(data[d].values.sum())

aggr_array1 = []
for d in date_cols:
    aggr_array1.append(dataN[d].values.sum())
aggr_array2 = []
for d in date_cols:
    aggr_array2.append(dataE[d].values.sum())
aggr_array3= []
for d in date_cols:
    aggr_array3.append(dataW[d].values.sum())
aggr_array4 = []
for d in date_cols:
    aggr_array4.append(dataS[d].values.sum())
date_range = pd.date_range(start='2018-04-01', end='2021-06-01',freq = 'm')

daily_time_series_df = pd.DataFrame(data=aggr_array, columns=['Sales'], index=date_range)
fig = make_subplots(rows=6, cols=1, subplot_titles=("Cummulative","North","East","West","South","Region Wise"))
fig.add_trace(go.Scatter(x=date_range, y=aggr_array, showlegend=False,
                    mode='lines', name="Aggregate Sales", marker=dict(color="darkgreen")), row = 1, col=1)
fig.add_trace(go.Scatter(x=date_range, y=aggr_array1, showlegend=False,
                    mode='lines', name="North", marker=dict(color="orange")), row = 2, col=1)
fig.add_trace(go.Scatter(x=date_range, y=aggr_array2, showlegend=False,
                    mode='lines', name="East", marker=dict(color="hotpink")), row = 3, col=1)
fig.add_trace(go.Scatter(x=date_range, y=aggr_array3, showlegend=False,
                    mode='lines', name="West", marker=dict(color="green")), row = 4, col=1)
fig.add_trace(go.Scatter(x=date_range, y=aggr_array4, showlegend=True,
                    mode='lines', name="South", marker=dict(color="blue")), row = 5, col=1)
fig.add_trace(go.Scatter(x=date_range, y=aggr_array1, showlegend=True,
                    mode='lines', name="North", marker=dict(color="orange")), row = 6, col=1)
fig.add_trace(go.Scatter(x=date_range, y=aggr_array2, showlegend=True,
                    mode='lines', name="East", marker=dict(color="hotpink")), row = 6, col=1)
fig.add_trace(go.Scatter(x=date_range, y=aggr_array3, showlegend=True,
                    mode='lines', name="West", marker=dict(color="green")), row = 6, col=1)
fig.add_trace(go.Scatter(x=date_range, y=aggr_array4, showlegend=False,
                    mode='lines', name="South", marker=dict(color="blue")), row = 6, col=1)

fig.update_layout(title="Aggregate Sales",height=1200, width=1000)
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Sales in Units", row=1, col=1)
fig.update_yaxes(title_text="Sales in Units", row=2, col=1)
fig.show()


###Seasonal Decomposition as Additive Model

In [ ]:
series = daily_time_series_df['Sales']
result = seasonal_decompose(series, model='additive')
result_list = [result.trend, result.seasonal, result.resid, result.observed]

fig = make_subplots(rows=len(result_list), cols=1, subplot_titles=("Trend", "Seasonal", "Residual", "Observed"))

for i, res in enumerate(result_list):
    fig.add_trace(go.Scatter(x=date_range, y=res, showlegend=True, mode='lines'), row=i + 1, col=1)

                  
fig.update_layout(title="Seasonal Decomposition Additive", height=1000, width=1000)
fig.update_xaxes(title_text="Date", row=4, col=1)
fig.update_yaxes(title_text="Value", row=1, col=1)
fig.update_yaxes(title_text="Value", row=2, col=1)
fig.update_yaxes(title_text="Value", row=3, col=1)
fig.update_yaxes(title_text="Value", row=4, col=1)                
fig.show()

###Plot Name??

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=date_range[:], y=result_list[1][:], showlegend=True, mode='lines'))
fig.update_layout(title="Seasonal Component", xaxis_title="Date", yaxis_title="Units/Value", height=500, width=800)
fig.show()

###Observed Sales and Trend Line

In [ ]:
X_values = range(len(daily_time_series_df))
coeffs = np.polyfit(X_values, daily_time_series_df['Sales'].values, 5)
poly_eqn = np.poly1d(coeffs)
y_hat = poly_eqn(X_values)

daily_time_series_df['sales_trend'] = y_hat

# since there are some nan values at the beginning and end lets take those values out.
decomp_trend = result.trend[3:-3]
coeffs = np.polyfit(X_values[3:-3], decomp_trend, 5)
poly_eqn = np.poly1d(coeffs)
y_hat = poly_eqn(X_values)

daily_time_series_df['sales_trend_decomp'] = y_hat
fig = go.Figure()
fig.add_trace(go.Scatter(x=daily_time_series_df.index, y=daily_time_series_df['Sales'].values,
                             showlegend=True, mode='lines'))
fig.add_trace(go.Scatter(x=daily_time_series_df.index, y=daily_time_series_df['sales_trend'].values, mode='lines',
                        showlegend=True, marker=dict(color="black"), name='observed_trend'))
fig.add_trace(go.Scatter(x=daily_time_series_df.index, y=daily_time_series_df['sales_trend_decomp'].values, mode='lines',
                        showlegend=True, marker=dict(color="orange"), name = 'decomposed trend'))
fig.update_layout(
    title="Observed Sales and trend-line", xaxis_title="Date", yaxis_title="Unit Sales", height=500, width=1000)
fig.show()

###Seasonal Decomposition as Multiplicative Model

In [ ]:
series = daily_time_series_df['Sales']
result = seasonal_decompose(series, model='multiplicative')
result_list = [result.trend, result.seasonal, result.resid, result.observed]

fig = make_subplots(rows=len(result_list), cols=1, subplot_titles=("Trend", "Seasonal", "Residual", "Observed"))

for i, res in enumerate(result_list):
    fig.add_trace(go.Scatter(x=date_range, y=res, showlegend=True, mode='lines'), row=i + 1, col=1)

                  
fig.update_layout(title="Seasonal Decomposition Multiplicative", height=1000, width=1000)
fig.update_xaxes(title_text="Date", row=4, col=1)
fig.update_yaxes(title_text="Value", row=1, col=1)
fig.update_yaxes(title_text="Value", row=2, col=1)
fig.update_yaxes(title_text="Value", row=3, col=1)
fig.update_yaxes(title_text="Value", row=4, col=1)                
fig.show()